In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from exod.utils.path import data_results
import pandas as pd
from glob import glob
import re
from itertools import combinations

from astropy.visualization import hist
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
from scipy.stats import skew, kurtosis
from tqdm import tqdm

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, HDBSCAN
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 20)

In [ ]:
# Load Lightcurves
regex = re.compile(r'\d{10}')
all_res = []
for f in glob('../data/results_combined/t_5_500_obs/*/*/*lcs.csv'):
    obsid = regex.findall(f)[0]
    df = pd.read_csv(f)
    print(f'obsid={obsid}')
    plt.figure(figsize=(15,3))
    plt.plot(df['time'], df['n_0'], color='black', label='Observed (n)', lw=1.0)
    plt.plot(df['time'], df['mu_0'], color='red', label=r'Expected ($\mu$)', lw=1.0)
    plt.legend()
    plt.show()
    if df.shape[1] > 7:
        n = int((df.shape[1] - 7) / 4)
        print(f'n={n}')
        for i in range(1,n):
            plt.figure(figsize=(15,3))
            plt.plot(df['time'], df[f'n_{i}'], color='black', label='Observed (n)', lw=1.0)
            plt.plot(df['time'], df[f'mu_{i}'], color='red', label=r'Expected ($\mu$)', lw=1.0)
            plt.legend()
            plt.show()
            
                
            
            
    
    
    

In [ ]:
# Load Lightcurves
regex = re.compile(r'\d{10}')
all_res = []
for f in glob('../data/results_combined/t_5_500_obs/*/*/*lcs.csv'):
    print(f)
    obsid = regex.findall(f)[0]
    df = pd.read_csv(f)
    plt.figure(figsize=(15,3))
    plt.plot(df['time'], df['B_peak_0'], label='B_Peak', lw=1.0)
    plt.plot(df['time'], df['B_eclipse_0'], label=r'B_eclipse', lw=1.0)
    plt.yscale('log')
    plt.legend()
    plt.show()

In [ ]:
# Detected Regions
dfs = []
for f in glob('../data/results_combined/t_5_500_obs/*/*/*regions.csv'):
    obsid = regex.findall(f)[0]
    df = pd.read_csv(f)
    df['obsid'] = obsid
    #df = df.T  
    #df['obsid'] = obsid
    dfs.append(df)

df_regions = pd.concat(dfs, ignore_index=True)
df_regions


In [ ]:
df_regions

In [ ]:
tab_xmm = Table.read('../data/util/4XMM_slim_DR13cat_v1.0.fits')
skycoord_xmm = SkyCoord(ra=tab_xmm['SC_RA'], dec=tab_xmm['SC_DEC'], unit=u.deg)
sky_coords = SkyCoord(ra=df_regions['ra_deg'].values, dec=df_regions['dec_deg'].values, unit='deg', frame='icrs')

cmatch = sky_coords.match_to_catalog_sky(skycoord_xmm)
tab_cmatch = Table(cmatch)
tab_cmatch.rename_columns(names=tab_cmatch.colnames, new_names=['idx', 'sep2d', 'dist3d'])
tab_cmatch['sep2d_arcsec'] = tab_cmatch['sep2d'].to(u.arcsec)
tab_cmatch['idx_orig'] = np.arange(len(tab_cmatch))

tab_xmm_cmatch = tab_xmm[tab_cmatch['idx']]
tab_xmm_cmatch['SEP'] = tab_cmatch['sep2d']
tab_xmm_cmatch['SEP_ARCSEC'] = tab_xmm_cmatch['SEP'] * 3600
tab_xmm_cmatch

In [ ]:
tab_closest

In [ ]:
for i, row in df_regions.iterrows():
    obsid   = row['obsid']
    label   = row['label']
    ra      = row['ra']
    dec     = row['dec']
    ra_deg  = row['ra_deg']
    dec_deg = row['dec_deg']

    csv_lc  = glob(f'../data/results_combined/t_5_500_obs/{obsid}/*/lcs.csv')
    csv_evt = glob(f'../data/results_combined/t_5_500_obs/{obsid}/*/evt_info.csv')
    img_png = glob(f'../data/results_combined/t_5_500_obs/{obsid}/*/*detection_img.png')

    if len(img_png) > 1:
        print('MULTIPLE SUBSETS!!!')
    
    df_lc = pd.read_csv(csv_lc[0])
    df_evt = pd.read_csv(csv_evt[0])
    im     = plt.imread(img_png[0])
    
    date  = df_evt.iloc[4].iloc[1]
    obj   = df_evt.iloc[5].iloc[1]

    tab_closest = tab_xmm_cmatch[i]
    c_ra      = tab_closest['SC_RA']
    c_dec     = tab_closest['SC_DEC']
    c_webpage = tab_closest['WEBPAGE_URL']    
    c_var     = tab_closest['SC_VAR_FLAG']
    c_sep     = tab_closest['SEP_ARCSEC']
    c_flux_8  = tab_closest['SC_EP_8_FLUX']

    esasky = f'https://sky.esa.int/esasky/?target={ra_deg}-{dec_deg}&hips=XMM-Newton+EPIC+color&fov=0.25&cooframe=J2000&sci=true&lang=en'
    simbad = f'https://simbad.cds.unistra.fr/simbad/sim-basic?Ident={ra_deg}+{dec_deg}'
    print(f'{i}/{len(df_regions)} | {obsid} | {obj} | {date} | {ra} {dec} {ra_deg} {dec_deg}')
    print(f'Closest DR13 ({c_sep:.2f}"): {c_ra} {c_dec} | SC_VAR_FLAG={c_var} | SC_EP_8_FLUX (0.2-12.0) = {c_flux_8:.3e}')
    print(f'{c_webpage}')
    print(f'{esasky}')
    print(f'{simbad}')
    plt.figure(figsize=(15,3))
    n = label - 1
    plt.plot(df_lc['time'], df_lc[f'n_{n}'], color='black', label='Observed (n)', lw=1.0)
    plt.plot(df_lc['time'], df_lc[f'mu_{n}'], color='red', label=r'Expected ($\mu$)', lw=1.0)
    plt.legend()
    #plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(15,15))
    plt.imshow(im)
    plt.axis('off')
    #plt.tight_layout()
    plt.show()

In [ ]:

for i, r in df_.iterrows():
    obsid = r['obsid']
    src   = r['src']
    
    df_evt = pd.read_csv(f'../data/results_combined/t_25s_5k_obs/{obsid}/evt_info.csv') 
    df_reg = pd.read_csv(f'../data/results_combined/t_25s_5k_obs/{obsid}/regions.csv') 
    df_lc  = pd.read_csv(f'../data/results_combined/t_25s_5k_obs/{obsid}/lcs.csv')
    
    # df_lc = df_lc[~df_lc['bti']]
    df_lc.loc[df_lc['bti'], src] = np.nan

    date  = df_evt.iloc[4].iloc[1]
    obj   = df_evt.iloc[5].iloc[1]

    # Crossmatch info
    tab_closest = tab_xmm_cmatch[i]
    c_ra      = tab_closest['SC_RA']
    c_dec     = tab_closest['SC_DEC']
    c_webpage = tab_closest['WEBPAGE_URL']    
    c_var     = tab_closest['SC_VAR_FLAG']
    c_sep     = tab_closest['SEP']
    c_flux_8  = tab_closest['SC_EP_8_FLUX']

    try:
        reg     = df_reg.iloc[int(src[-1])]
        ra      = reg['ra']
        dec     = reg['dec']
        ra_deg  = reg['ra_deg']
        dec_deg = reg['dec_deg']
    except Exception as e:
        print(f'woowie! {e}')
        pass

    esasky = f'https://sky.esa.int/esasky/?target={ra_deg}-{dec_deg}&hips=XMM-Newton+EPIC+color&fov=0.25&cooframe=J2000&sci=true&lang=en'
    f.write(f'{obsid} | {src} | {obj} | {date} | {ra} {dec} {ra_deg} {dec_deg}\n')
    f.write(f'Closest DR13 ({c_sep*3600:.2f}"): {c_ra} {c_dec} | SC_VAR_FLAG={c_var} | SC_EP_8_FLUX (0.2-12.0) = {c_flux_8:.3e}\n')
    f.write(f'<a href="{c_webpage}">{c_webpage}</a>\n')
    f.write(f'<a href="{esasky}">{esasky}</a>\n')
    f.write(f'<img src="lcs/{i}.png">\n')
    
    plt.figure(figsize=(15,3))
    plt.step(df_lc['time'], df_lc[src], label=f'{obsid} | {src}', lw=1.0, color='black')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'lcs/{i}.png')
    plt.close()
    #plt.show()

f.write("</pre></body></html>")


In [ ]:
"""
# Clustering with HDBScan
df_features = pd.DataFrame(all_res)
cols = ['mean', 'std', 'var', 'cv', 'median', 'mode', 'skew', 'kurt', 'min', 'max', 'len']
df_features[cols] = df_features[cols].apply(lambda x: np.log1p(x))
df_features = df_features[~df_features.isna().any(axis=1)]

X = df_features[cols]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

hdbscan = HDBSCAN(min_cluster_size=20)
df_features['cluster'] = hdbscan.fit_predict(X_scaled)
"""

In [ ]:
df_features['cluster'].value_counts()

In [ ]:
df_features[df_features['cluster'] == 7]

In [ ]:
obsid = '0810811801'
src   = '0'
df_lc = pd.read_csv(f'../data/results_combined/t_25s_5k_obs/{obsid}/lcs.csv')
df_lc = df_lc[~df_lc['bti']]
#print(df_lc)
df_lc.plot(x='time', y='src_0')

In [ ]:
# HDBScan
# TSNE

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(X)
X_embedded.shape